In [1]:
import cv2
import numpy as np

# KCF Object Tracking

In [2]:
#initialize the ‘KCF’ tracker object
tracker = cv2.TrackerKCF_create() 

#video = cv2.VideoCapture(r'C:\Users\Naim Rihan\Project\watch.mp4')

#initialize the camera
video= cv2.VideoCapture(0)

#fetch the first frame of the video
ok,frame=video.read() 

#initialize the ‘selectROI’ function with the first frame of the video which we fetched
#and store its value in the ‘bbox’ variable.
bbox = cv2.selectROI(frame)

#initialize the tracker (using ‘init’) with the frame (in which we selected our region of interest)
#and position (bbox) of the object to be tracked.
ok = tracker.init(frame,bbox)

#Initialize a while loop that loops through the frames of our stream
while True:
    #fetch the frames of the video along with a flag parameter(‘ok’)
    #which informs if the frame fetching process was successful or not.
   ok,frame=video.read()

    #If the flag parameter is false the execution stops i.e. if the stream is not fetched properly the execution is stopped.
   if not ok:
        break
    #pass a new consecutive frame with every iteration of the loop.
    #It returns two variables, first is a flag parameter that informs if the tracking process was successful or not
    #and the second returns the position of the tracked object in the frame if and only if the first parameter was true.
    
    
   ok,bbox=tracker.update(frame)
   if ok: #If the ‘ok’ flag is true this block is executed. We fetched the position of the object in the ‘bbox’ variable, here we initialize the x,y coordinates and the values of width and height. Next, we use the OpenCV ‘rectangle’ function to put a bounding box around the detected object in consecutive frames of the video.
        (x,y,w,h)=[int(v) for v in bbox]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2,1)
        
   else: #If the tracker is unable to track the selected ROI or faces any errors, this block of code prints ‘Error’ on the video frames.
        cv2.putText(frame,'Error',(100,0),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
        
   cv2.imshow('Tracking',frame) #Showing the stream frames on a separate window using the ‘cv2.imshow’ function
   if cv2.waitKey(1) & 0XFF==27: #If the user clicks the ‘escape’ button execution stops
        break
cv2.destroyAllWindows() #close all lingering windows if there are any.

# CSRT Object Tracking

In [3]:
tracker = cv2.TrackerCSRT_create()
video= cv2.VideoCapture(0)
ok,frame=video.read()
bbox = cv2.selectROI(frame)
ok = tracker.init(frame,bbox)
while True:
    ok,frame=video.read()
    if not ok:
        break
    ok,bbox=tracker.update(frame)
    if ok:
        (x,y,w,h)=[int(v) for v in bbox]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2,1)
    else:
        cv2.putText(frame,'Error',(100,0),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.imshow('Tracking',frame)
    if cv2.waitKey(1) & 0XFF==27:
        break
cv2.destroyAllWindows()

# Mean Shift Object Tracker

In [6]:
cap = cv2.VideoCapture(0)
ret,frame=cap.read()
x,y,w,h = cv2.selectROI(frame)
track_window = (x, y, w, h)
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
   ret, frame = cap.read()
   if ret == True:
       hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
       dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
       ret, track_window = cv2.meanShift(dst, track_window, term_crit)
       x,y,w,h = track_window
       img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
       cv2.imshow('img2',img2)
       k = cv2.waitKey(30) & 0xff
       if k == 27:
           break
   else:
       break
cv2.destroyAllWindows()

# Don't Touch This!

In [8]:
import requests
import cv2
import numpy as np
import imutils

url = "http://192.168.112.140:8080/shot.jpg"

while True:
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    img = imutils.resize(img, width=1000, height=1800)
    cv2.imshow("Android_cam", img)
  
    # Press Esc key to exit
    if cv2.waitKey(1) == 27:
        break
  
cv2.destroyAllWindows()

# Cam Shift Object Tracker

In [5]:
cap = cv2.VideoCapture(0)
ret,frame=cap.read()
x,y,w,h = cv2.selectROI(frame)
track_window = (x, y, w, h)
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
   ret, frame = cap.read()
   if ret == True:
       hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
       dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
       ret, track_window = cv2.CamShift(dst, track_window, term_crit)
       pts = cv2.boxPoints(ret)
       pts=np.int0(pts)
       img2 = cv2.polylines(frame, [pts], True, 255,2)
       cv2.imshow('img2',img2)
       k = cv2.waitKey(30) & 0xff
       if k == 27:
           break
   else:
       break
cv2.destroyAllWindows()